In [14]:
import xarray as xr
import numpy as np
import dask
from distributed import Client
import dask_jobqueue
#import daskdataframe as dd
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
import xskillscore as xs
import cartopy as cart
%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)

In [15]:
#client = Client()
LAT        = 50.42
LON        = 228.8
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
temp_path  = '/global/scratch/users/harsha/LENS/cesm2/cvals/global/temps/'
qpath      = '/global/scratch/users/harsha/LENS/cesm2/cvals/global/quantiles/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/global/rmse/'
cvals_us   = '/global/scratch/users/harsha/LENS/cesm2/cvals/conus/'
loc        = 'columbia'
cvals1     = '/global/scratch/users/harsha/LENS/cesm2/tmax_mem'
mae_path   = '/global/scratch/users/harsha/LENS/cesm2/cvals/global/mae/'

In [16]:
#client

In [19]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192'] 
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=20, walltime='6:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(10)

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45471 instead
  warnings.warn(


In [20]:
cluster

In [6]:
tw2_path = temp_path+'/twJul30_2020_2050_global.zarr'
tgw  = xr.open_zarr(tw2_path)
data = tgw.TREFHTMX
data

<xarray.DataArray 'TREFHTMX' (lat: 192, lon: 288, mtime: 3100)>
dask.array<open_dataset-e309dde6a65c49a4ee445b73d7ddd9a3TREFHTMX, shape=(192, 288, 3100), dtype=float32, chunksize=(192, 288, 31), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (mtime) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    time     (mtime) object dask.array<chunksize=(3100,), meta=np.ndarray>
Dimensions without coordinates: mtime
Attributes:
    cell_methods:  time: maximum
    long_name:     Maximum reference height temperature over output period
    units:         K

In [21]:
iterations = 10
samples    = 3100
quantiles  = np.linspace(0,1,1000)
CIs        = [0.025, 0.975]

In [25]:
%%time
# you will probably have to play around with chunksize on lrc
# keep in mind that data grows by a factor of iterations*samples
# when computing the quantiles
data  = data.chunk(mtime=-1, lon=50, lat=50)
data

CPU times: user 866 µs, sys: 0 ns, total: 866 µs
Wall time: 820 µs


<xarray.DataArray 'TREFHTMX' (lat: 192, lon: 288, mtime: 3100)>
dask.array<rechunk-merge, shape=(192, 288, 3100), dtype=float32, chunksize=(50, 50, 3100), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (mtime) int64 dask.array<chunksize=(3100,), meta=np.ndarray>
    time     (mtime) object dask.array<chunksize=(3100,), meta=np.ndarray>
Dimensions without coordinates: mtime
Attributes:
    cell_methods:  time: maximum
    long_name:     Maximum reference height temperature over output period
    units:         K

In [26]:
%%time
rand_indices_2d = np.random.randint(0, data.mtime.size, (iterations, samples))
rand_indices_da = xr.DataArray(rand_indices_2d, dims=['iteration', 'sample'])
data_samples    = data[:,:,rand_indices_da]
data_quantiles  = data_samples.quantile(quantiles, dim='sample')
data_quantiles

CPU times: user 1.61 s, sys: 55.1 ms, total: 1.67 s
Wall time: 1.64 s


/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/dask/utils.py:295: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  return self.fn(key)


<xarray.DataArray 'TREFHTMX' (quantile: 1000, lat: 192, lon: 288, iteration: 10)>
dask.array<transpose, shape=(1000, 192, 288, 10), dtype=float64, chunksize=(1000, 50, 50, 10), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * quantile  (quantile) float64 0.0 0.001001 0.002002 ... 0.998 0.999 1.0
Dimensions without coordinates: iteration

In [ ]:
%%time
# by default we have to rename quantile or calling the quantile function()
# again generates an error since it expects to create a new dimension
# called quantile
data_quantiles  = data_quantiles.rename({'quantile':'sample_quantile'})
data_CIs        = data_quantiles.quantile(CIs, dim='iteration')
data_CIs        = data_CIs.rename({'quantile':'confidence_interval',
                                   'sample_quantile':'quantile'})
data_CIs        = data_CIs.compute()

In [11]:
data_quantiles

<xarray.DataArray 'TREFHTMX' (sample_quantile: 1000, lat: 192, lon: 288,
                              iteration: 100)>
dask.array<transpose, shape=(1000, 192, 288, 100), dtype=float64, chunksize=(1000, 50, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * lat              (lat) float64 -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon              (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * sample_quantile  (sample_quantile) float64 0.0 0.001001 ... 0.999 1.0
Dimensions without coordinates: iteration

In [12]:
%%time
data_CIs.isel(lat=2, lon=3, confidence_interval=0).plot()
data_CIs.isel(lat=2, lon=3, confidence_interval=1).plot()


KeyboardInterrupt



In [13]:
data_CIs.isel(confidence_interval=0, quantile=90).plot()

Task exception was never retrieved
future: <Task finished name='Task-74' coro=<Client._gather.<locals>.wait() done, defined at /global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/client.py:2002> exception=AllExit()>
Traceback (most recent call last):
  File "/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/client.py", line 2011, in wait
    raise AllExit()
distributed.client.AllExit

KeyboardInterrupt

